A survey of egraphs for compilers

What is an egraph?
We can rewrite terms. It is not so hard to share children. Rewrite high up in the term (root) don't need much extra space.
aegraph does all rewrites at root. Which is smart.
Sharing parents is harder. (although with a focussed zipper, it's the same thing.) Rewrites low in the tree 
An indirection solves the issue.

Equational Reasoning
High school algebra
vector algebra
Matrix algebra

Sums
Integrals
einstein notation
tensor algebra


Peephole optimizers
Rewrites


Denali

PEGs


RVSDG



Diospyros
Isaria


Glenside
Categorical Rewriting


SPORES
Tensat
Babble


Isle
Cranelift https://github.com/bytecodealliance/rfcs/pull/27


SqEQ
lean-egg



Sam Coward
https://arxiv.org/abs/2303.01839 
Automating Constraint-Aware Datapath Optimization using E-Graphs


Backwards search for egg is a good start

Yihong's list of projects that use equality saturation
https://necessary-taker-84c.notion.site/Projects-that-use-equality-saturation-9b298720aa724f8b8a72001398e88d9f


https://egraphs.org/meeting/2024-09-19-isaria 
https://dl.acm.org/doi/10.1145/3617232.3624873 Automatic Generation of Vectorizing Compilers for Customizable Digital Signal Processors
https://github.com/sgpthomas/flexible-isa
https://github.com/sgpthomas/egg-stats
https://github.com/sgpthomas/comp-gen


Latent Idiom Recognition for a Minimalist Functional Array Language using Equality Saturation https://arxiv.org/abs/2312.17682

https://egraphs.org/meeting/2024-04-18-sparse-tensor-diff
https://arxiv.org/pdf/2303.07030  ∇SD: Differentiable Programming for Sparse Tensors



Scheduling
What is it?

https://github.com/egraphs-good/egg/discussions/106 Representing Loops within Egg

https://github.com/miestrode/bigbrain bigbrain logic circuit optimizer
https://github.com/miestrode/ekege

https://github.com/khaki3/acc-saturator  Equality saturation for OpenACC/OpenMP in C. https://arxiv.org/pdf/2306.13002 ACC Saturator: Automatic Kernel Optimization for
Directive-Based GPU Code

https://github.com/RobinKa/egga  E-Graph Geometric Algebra (EGGA)

https://gitee.com/overshiki/egraph_circuit?skip_mobile=true C++ egraph

https://github.com/jafioti/luminal luminal. Using egraphs somewehre?

https://github.com/AD1024/flexmatch Flexible Matching via Equality Saturation Flexible Matching: Explore and reveal accelerator-supported workloads in Deep Learning Models


https://www.cs.princeton.edu/~appel/papers/ssafun.pdf

In [ ]:
import kdrag as kd


https://ztatlock.net/pubs/2009-popl-equality-saturation-optimizations-egraphs.pdf https://rosstate.org/publications/eqsat/

Control flow graph (CFG) is just like, a thing. Its denotational semantics are a bit confused.

Egraphs like talking about functions. How to encode something cfg like into functions?

Dataflow graphs are basically just a DAG functional representation. So that intra block dataflow is not really a problem.

Being in SSA make one close to purely functional. YOu never assign to the sam name twice (although let binding shadowing could also model this purely functionally).

SSA Phi nodes are slightly bizarre. Gated Phi nodes make explicit the condition on which the merging happens. This makes them more like a purely functional if-then-else.

One can consider using streams as a functional representation of loop behavior. There are operations like tail, head, other on the stream.

PEGs have some clues on how to treat summation symbols 
 in an egraph. Surely you can write a sum function asa CFG. Partial Sums / indefinite integrals rather than closed sum, definite integrals.

Is there a relationship between pegs and timely dataflow?

You only Grep once by koppel - uses PEG-like representation for semantic code search.

representing loops within egg

RVSDG https://github.com/jameysharp/optir/

Loops in egraphs and Landin’s knot.

Equality Saturation: a New Approach to Optimization

Quiche quiche python egraph for manipulating python AST

In [1]:
from __future__ import annotations
from egglog import *


class Num(Expr):
    def __init__(self, value: i64Like) -> None: ...

    @classmethod
    def var(cls, name: StringLike) -> Num:  ...

    def __add__(self, other: Num) -> Num: ...

    def __mul__(self, other: Num) -> Num: ...

egraph = EGraph()

expr1 = egraph.let("expr1", Num(2) * (Num.var("x") + Num(3)))
expr2 = egraph.let("expr2", Num(6) + Num(2) * Num.var("x"))

@egraph.register
def _num_rule(a: Num, b: Num, c: Num, i: i64, j: i64):
    yield rewrite(a + b).to(b + a)
    yield rewrite(a * (b + c)).to((a * b) + (a * c))
    yield rewrite(Num(i) + Num(j)).to(Num(i + j))
    yield rewrite(Num(i) * Num(j)).to(Num(i * j))

egraph.saturate()


In [2]:
egraph.check(eq(expr1).to(expr2))
egraph.extract(expr1)

(Num(2) * Num.var("x")) + Num(6)

In [5]:
egraph = EGraph()
dir(egraph)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__egg_decls__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_init__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_decls',
 '_command_to_egg',
 '_egraph',
 '_facts_to_check',
 '_flatted_deps',
 '_register_commands',
 '_repr_html_',
 '_repr_mimebundle_',
 '_run_extract',
 '_run_schedule',
 '_state',
 '_state_stack',
 '_token_stack',
 'as_egglog_string',
 'check',
 'check_fail',
 'class_',
 'constant',
 'current',
 'display',
 'eval',
 'extract',
 'extract_multiple',
 'function',
 'graphviz',
 'graphviz_svg',
 'include',
 'input',
 'let',
 'method',
 'modules',
 'output',
 'pop',
 